In [ ]:
# ===========================================================
# Notebook 7 — Reference & Reuse from Notebooks 01 & 02
# ===========================================================
# This cell:
# 1) Attempts to execute prior notebooks to pull in any helper functions (metrics, plotting, etc.)
# 2) Provides safe fallbacks for regression & classification metrics if those notebooks
#    don't expose functions in the global namespace.
# 3) Prints a short "References" section for report traceability.
# -----------------------------------------------------------

import os, builtins, types, inspect, sys
import numpy as np

# --- 1) Try to execute prior notebooks so their functions land in the global namespace ---
possible_paths = [
    "/mnt/data/01_regression_evaluation.ipynb",
    "/mnt/data/02_classification_evaluation.ipynb",
    "01_regression_evaluation.ipynb",
    "02_classification_evaluation.ipynb",
    "/mnt/data/01_linear_regression (1).ipynb",  # just in case you still want this one
]

# Jupyter magic runner, if available  
def _run_notebook(nb_path):
    try:
        get_ipython().run_line_magic("run", f'"{nb_path}"')
        return True
    except Exception as e:
        print(f"[REF] Could not run: {nb_path} -> {e}")
        return False

ran_any = False
for p in possible_paths:
    if os.path.exists(p) and p.lower().endswith(".ipynb"):
        ok = _run_notebook(p)
        ran_any = ran_any or ok

# --- 2) Safe fallbacks (only define if missing) ------------------------------------------
def _maybe_define(name, fn):
    if not hasattr(builtins, name) and name not in globals():
        globals()[name] = fn

# Regression metrics fallbacks
_maybe_define = _maybe_define  # alias

_maybe_define("mse", lambda y_true, y_pred: float(np.mean((np.asarray(y_true) - np.asarray(y_pred))**2)))
_maybe_define("mae", lambda y_true, y_pred: float(np.mean(np.abs(np.asarray(y_true) - np.asarray(y_pred)))))
def _rmse(y_true, y_pred): 
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    return float(np.sqrt(np.mean((y_true - y_pred)**2)))
_maybe_define("rmse", _rmse)
def _r2(y_true, y_pred):
    y_true = np.asarray(y_true); y_pred = np.asarray(y_pred)
    ss_res = float(np.sum((y_true - y_pred)**2))
    ss_tot = float(np.sum((y_true - np.mean(y_true))**2))
    return 1.0 - ss_res/ss_tot if ss_tot != 0 else 0.0
_maybe_define("r2", _r2)

# Classification metrics fallbacks (vectorized; robust for edge cases)
def _confusion_matrix_np(y_true, y_pred):
    y_true = np.asarray(y_true).astype(int); y_pred = np.asarray(y_pred).astype(int)
    # binary only
    tn = int(np.sum((y_true == 0) & (y_pred == 0)))
    fp = int(np.sum((y_true == 0) & (y_pred == 1)))
    fn = int(np.sum((y_true == 1) & (y_pred == 0)))
    tp = int(np.sum((y_true == 1) & (y_pred == 1)))
    return np.array([[tn, fp], [fn, tp]], dtype=int)
_maybe_define("confusion_matrix_np", _confusion_matrix_np)

def _accuracy(y_true, y_pred):
    y_true = np.asarray(y_true).astype(int); y_pred = np.asarray(y_pred).astype(int)
    return float(np.mean(y_true == y_pred))
_maybe_define("accuracy_np", _accuracy)

def _precision(y_true, y_pred, zero_division=0.0):
    cm = _confusion_matrix_np(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    denom = (tp + fp)
    return float(tp/denom) if denom else float(zero_division)
_maybe_define("precision_np", _precision)

def _recall(y_true, y_pred, zero_division=0.0):
    cm = _confusion_matrix_np(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    denom = (tp + fn)
    return float(tp/denom) if denom else float(zero_division)
_maybe_define("recall_np", _recall)

def _f1(y_true, y_pred, zero_division=0.0):
    p = _precision(y_true, y_pred, zero_division=zero_division)
    r = _recall(y_true, y_pred, zero_division=zero_division)
    denom = (p + r)
    return float(2*p*r/denom) if denom else float(zero_division)
_maybe_define("f1_np", _f1)

def _tpr_fpr(y_true, y_pred):
    cm = _confusion_matrix_np(y_true, y_pred)
    tn, fp, fn, tp = cm.ravel()
    tpr = float(tp/(tp+fn)) if (tp+fn) else 0.0
    fpr = float(fp/(fp+tn)) if (fp+tn) else 0.0
    return tpr, fpr
_maybe_define("tpr_fpr_np", _tpr_fpr)

# Small helper to binarize probabilities by threshold (used across notebooks)
def _binarize(p, thr=0.5):
    return (np.asarray(p) >= float(thr)).astype(int)
_maybe_define("binarize", _binarize)

# --- 3) Human-readable references you can print in your report/notebook ------------------
refs = [
    "01_regression_evaluation.ipynb — Regression metrics & analysis (Task 1 follow-up).",
    "02_classification_evaluation.ipynb — Classification metrics & analysis (Task 3 follow-up).",
]
print("\n=== References used in this notebook ===")
for i, r in enumerate(refs, 1):
    print(f"[{i}] {r}")

print("\nLoaded prior notebooks:", "YES" if ran_any else "NO (using local fallbacks)")

# After this cell:
# - Your previously-defined functions (if any) from 01/02 will already be in the global namespace.
# - Missing pieces are covered by the safe fallbacks above.
# - Continue with Notebook 7’s unified evaluation code (metrics/curves/threshold analysis).



=== References used in this notebook ===
[1] 01_regression_evaluation.ipynb — Regression metrics & analysis (Task 1 follow-up).
[2] 02_classification_evaluation.ipynb — Classification metrics & analysis (Task 3 follow-up).

Loaded prior notebooks: NO (using local fallbacks)
